In [1]:
import pandas as pd
from datetime import timedelta
# data
scan_data = pd.read_csv('http://95.99.70.232:8000/shortapi_data.csv', sep=',', index_col=0)
# Convert timestamp strings to datetime objects
scan_data['timestamp'] = pd.to_datetime(scan_data['timestamp'])
scans = scan_data.iloc[-1]

review_data = pd.read_csv('http://95.99.70.232:8000/googleapi_data.csv', sep=',', index_col=0)
# Convert timestamp strings to datetime objects
review_data['timestamp'] = pd.to_datetime(review_data['timestamp'])
reviews = review_data.iloc[-1]
reviews_count_old = 604

# Verify review integrity via saved logs
pre_scans_data = scan_data[['count', 'timestamp']]
result_data = []

for _, scan_row in pre_scans_data.iterrows():
    scan_timestamp = scan_row['timestamp']
    valid_passes = review_data[
        (review_data['timestamp'] >= scan_timestamp - timedelta(minutes=5)) &
        (review_data['timestamp'] <= scan_timestamp + timedelta(minutes=30))
    ]

    if not valid_passes.empty:
        for _, pass_row in valid_passes.iterrows():
            if pass_row['timestamp'] > scan_row['timestamp']:
                time_difference = pass_row['timestamp'] - scan_row['timestamp']
            else:
                time_difference = scan_row['timestamp'] - pass_row['timestamp']
            result_data.append({
                'scan_count': scan_row['count'],
                'pass_count': pass_row['count'],
                'scan_timestamp': scan_row['timestamp'],
                'pass_timestamp': pass_row['timestamp'],
                'time_difference': time_difference
            })

result_df = pd.DataFrame(result_data)
# row a
totals = scans['count']

print(f"Totaal aantal scans", totals)
print("Aantal reviews door Boostercards ",result_df.shape[0] ,int(reviews['count']))
result_df

Totaal aantal scans 60
Aantal reviews door Boostercards  3 624


,scan_count,pass_count,scan_timestamp,pass_timestamp,time_difference
0,53,620,2023-08-07 17:05:33,2023-08-07 17:20:12,0 days 00:14:39
1,57,621,2023-08-09 23:28:10,2023-08-09 23:28:10,0 days 00:00:00
2,60,623,2023-08-10 21:03:06,2023-08-10 21:05:03,0 days 00:01:57


In [5]:
review_data

,count,timestamp
0,619,2023-08-07 10:15:22
1,620,2023-08-07 17:20:12
2,621,2023-08-10 00:20:06
3,622,2023-08-10 16:15:07
4,623,2023-08-10 18:05:06


In [7]:
scan_data

,count,timestamp,c1_human,c1_ios,c1_android,c2_human,c2_ios,c2_android,c3_human,c3_ios,c3_android
0,40,2023-08-07 13:12:33,7,7,0,1,1,0,32,31,0
1,43,2023-08-07 13:13:33,7,7,0,1,1,0,35,33,0
2,44,2023-08-07 13:28:33,8,8,0,1,1,0,35,33,0
3,47,2023-08-07 13:30:44,9,9,0,1,1,0,37,35,0
4,53,2023-08-07 17:05:33,13,13,0,1,1,0,39,37,0
5,55,2023-08-07 18:28:29,14,14,0,1,1,0,40,38,0
6,57,2023-08-09 23:28:10,14,14,0,1,1,0,42,40,0
7,60,2023-08-10 18:10:06,14,14,0,2,2,0,44,42,0


In [5]:
import json
with open('config.json') as config_file:
    config = json.load(config_file)

input_string = config["name"]
output_string = input_string.replace(" ", "").lower()

filepath_google = f'/home/leytonserver/host/data/googleapi_data_{output_string}.csv'
print(filepath_google)

/home/leytonserver/host/data/googleapi_data_torossantiagodelft.csv


In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

with open('config.json') as config_file:
    config = json.load(config_file)

output_string = config["name"].replace(" ", "").lower()
filepath_short = f'/home/leytonserver/host/data/shortapi_data_{output_string}.csv'

def get(linkID):
    if linkID != "None":
        APIKEY = config["short"]
        url = f'https://api-v2.short.io/statistics/link/{linkID}'
        querystring = {
            "period": "total",
            "tzOffset": "0"
        }
        headers = {
            'accept': "*/*",
            'authorization': APIKEY
        }
        response = requests.request("GET", url, headers=headers, params=querystring)
        data = json.loads(response.text) 
        
        score_human = data["humanClicks"]

        score_ios = 0
        for os_item in data["os"]:
            if os_item["os"] == "iOS":
                score_ios = os_item["score"]
                break

        score_android = 0
        for os_item in data["os"]:
            if os_item["os"] == "Android":
                score_android = os_item["score"]
                break
    else:
        score_human = 0
        score_ios = 0
        score_android = 0
    
    return {'human': score_human, 'iOS': score_ios, 'Android': score_android}
get("lnk_3q7S_6th6d48vzTemIknHIjvi7")

{'human': 5, 'iOS': 4, 'Android': 0}

In [3]:
import json
with open('config.json') as config_file:
    config = json.load(config_file)

output_string = config["name"].replace(" ", "").lower()
filepath_google = f'/home/leytonserver/host/data/googleapi_data_{output_string}.csv'

filepath_google

'/home/leytonserver/host/data/googleapi_data_dekurk-lokaalproeffestijn.csv'